In [1]:
import sys
import os
import numpy as np
import pandas as pd
import re
import itertools
import tensorflow as tf
import string
from io import BytesIO
from tensorflow.contrib import learn
from collections import Counter
from time import time
import datetime
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

# Read the input dataset 
d = pd.read_csv("./consumer_complaints.csv", 
                usecols=('product','consumer_complaint_narrative'),
                dtype={'consumer_complaint_narrative': object})
# Only interested in data with consumer complaints
d=d[d['consumer_complaint_narrative'].notnull()]
d=d[d['product'].notnull()]
d.reset_index(drop=True,inplace=True)

c = d['product'].unique()



array(['Debt collection', 'Consumer Loan', 'Mortgage', 'Credit card',
       'Credit reporting', 'Student loan', 'Bank account or service',
       'Payday loan', 'Money transfers', 'Other financial service',
       'Prepaid card'], dtype=object)

In [49]:
c = d['product'].unique()
[i.split() for i in c]

AttributeError: 'numpy.ndarray' object has no attribute 'lower'

In [60]:
[word_data[0]]+list(d['product'].unique())
[i for i in vocab if i in vocab.keys()]


['opened',
 'account',
 'bank',
 'online',
 '15',
 'immediately',
 'transfer',
 'week',
 'later',
 'got',
 'letter',
 'copy',
 'address',
 'today',
 'hold',
 'returned',
 'information',
 'long',
 'want',
 'america',
 'gave',
 'money',
 'know',
 'receive',
 'tell',
 'just',
 've',
 'check',
 'year',
 'paid',
 'card',
 'longer',
 'credit',
 'use',
 'work',
 '100',
 'dollars',
 'pay',
 'bills',
 'allowed',
 'debit',
 'contacted',
 'time',
 'savings',
 'sent',
 'total',
 'month',
 'stating',
 'let',
 'date',
 'report',
 'received',
 'saying',
 'refused',
 'send',
 'signed',
 'direct',
 'deposit',
 'customer',
 'service',
 'set',
 'monthly',
 'spoke',
 'supervisor',
 'phone',
 'ago',
 'provide',
 'months',
 'went',
 'getting',
 'charges',
 'checks',
 'error',
 'using',
 'provided',
 'way',
 'accounts',
 'tried',
 'checking',
 'atm',
 'overdraft',
 'overdrawn',
 'explained',
 'department',
 'transactions',
 'trying',
 'banks',
 'financial',
 'having',
 'fraud',
 'branch',
 'manager',
 'secur

In [32]:

def clean_str(string):
    """
    Tokenization/string cleaning (partially modified)
    """
    string = re.sub(r"[^A-Za-z0-9()!?\'\`%$]", " ", string) # keep also %$ but removed comma
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " ( ", string)
    string = re.sub(r"\)", " ) ", string)
    string = re.sub(r"\?", " ? ", string)
    string = re.sub(r"\$", " $ ", string) #yes, isolate $
    string = re.sub(r"\%", " % ", string) #yes, isolate %
    string = re.sub(r"\s{2,}", " ", string)
    
    # fixing XXX and xxx like as word
    string = re.sub(r'\S*(x{2,}|X{2,})\S*',"",string)
    # removing non ascii
    string = re.sub(r'[^\x00-\x7F]+', "", string) 
    
    string = string.split()
    string= [w for w in string if not w in stop_words]
    string = " ".join(string)
    
    return string.strip().lower()

['called',
 'told',
 '00',
 'did',
 'calling',
 'said',
 'asked',
 'calls',
 'mortgage']

In [95]:
classTopics=[]
#stop_words = ['called','told','00',  'did', 'calling','said', 'asked','calls']
for u in d['product'].unique():
    a = d[d['product']==u]
    word_data=[]


    for message in a['consumer_complaint_narrative']:
        stop_words = ['called','told','00',  'did', 'calling','said', 'asked','calls'] + u.lower().split()
        word_data.append(clean_str(message))

    len(word_data)

    from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
    import guidedlda

    tf_vectorizer = CountVectorizer(max_df=0.95, min_df=0.05,
                                    max_features=4000,
                                    stop_words='english')
    tf = tf_vectorizer.fit_transform(word_data+list(d['product'].unique()))
    model = guidedlda.GuidedLDA(n_topics=4, n_iter=200, random_state=8, refresh=20)

    vocab = tf_vectorizer.vocabulary_
    vocab.keys()
    keysA = vocab.keys()
    contentsA = vocab.items()

    seed_topic_list = [u.lower().split()]
    seed_topics = {}
    for t_id, st in enumerate(seed_topic_list):
        for word in st:
            if word in vocab.keys():
                seed_topics[vocab[word]] = t_id

    model.fit(tf, seed_topics=seed_topics, seed_confidence=0.15)

    topicCont=[]
    for topicCt in range(4):
        n_top_words = 8
        inv_map = { v:k for k,v in contentsA }
        topic_word = model.topic_word_
        print(topic_word.shape)
        topWords = np.argsort(topic_word[topicCt])[:-(n_top_words+1):-1]
        topWordsRead =[]
        for i in topWords:
            topWordsRead +=[inv_map[i]]
        print(topWordsRead)
        topicCont+=[topWordsRead]
    classTopics+=[topicCont]

INFO:guidedlda:n_documents: 17563
INFO:guidedlda:vocab_size: 148
INFO:guidedlda:n_words: 374197
INFO:guidedlda:n_topics: 4
INFO:guidedlda:n_iter: 200
INFO:guidedlda:<0> log likelihood: -2527714
INFO:guidedlda:<20> log likelihood: -1851722
INFO:guidedlda:<40> log likelihood: -1814797
INFO:guidedlda:<60> log likelihood: -1806972
INFO:guidedlda:<80> log likelihood: -1803054
INFO:guidedlda:<100> log likelihood: -1799693
INFO:guidedlda:<120> log likelihood: -1798699
INFO:guidedlda:<140> log likelihood: -1796850
INFO:guidedlda:<160> log likelihood: -1796076
INFO:guidedlda:<180> log likelihood: -1796170
INFO:guidedlda:<199> log likelihood: -1795826


(4, 148)
['credit', 'account', 'report', 'company', 'information', 'letter', 'received', 'sent']
(4, 148)
['phone', 'number', 'company', 'pay', 'payment', 'received', 'account', 'time']
(4, 148)
['credit', 'account', 'report', 'letter', 'company', 'received', 'sent', 'information']
(4, 148)
['credit', 'company', 'phone', 'number', 'account', 'received', 'paid', 'information']


INFO:guidedlda:n_documents: 3689
INFO:guidedlda:vocab_size: 211
INFO:guidedlda:n_words: 129194
INFO:guidedlda:n_topics: 4
INFO:guidedlda:n_iter: 200
INFO:guidedlda:<0> log likelihood: -886764
INFO:guidedlda:<20> log likelihood: -690517
INFO:guidedlda:<40> log likelihood: -671054
INFO:guidedlda:<60> log likelihood: -665482
INFO:guidedlda:<80> log likelihood: -663787
INFO:guidedlda:<100> log likelihood: -662276
INFO:guidedlda:<120> log likelihood: -662072
INFO:guidedlda:<140> log likelihood: -661105
INFO:guidedlda:<160> log likelihood: -661308
INFO:guidedlda:<180> log likelihood: -661418
INFO:guidedlda:<199> log likelihood: -661983


(4, 211)
['payment', 'credit', 'car', 'payments', 'vehicle', 'paid', 'account', 'pay']
(4, 211)
['payment', 'account', 'bank', 'credit', 'received', 'late', 'pay', 'payments']
(4, 211)
['credit', 'vehicle', 'car', 'account', 'report', 'information', 'received', 'company']
(4, 211)
['car', 'company', 'payment', 'pay', 'credit', 'payments', 'time', 'paid']


INFO:guidedlda:n_documents: 14930
INFO:guidedlda:vocab_size: 298
INFO:guidedlda:n_words: 776968
INFO:guidedlda:n_topics: 4
INFO:guidedlda:n_iter: 200
INFO:guidedlda:<0> log likelihood: -5487660
INFO:guidedlda:<20> log likelihood: -4399407
INFO:guidedlda:<40> log likelihood: -4280635
INFO:guidedlda:<60> log likelihood: -4251662
INFO:guidedlda:<80> log likelihood: -4238497
INFO:guidedlda:<100> log likelihood: -4233600
INFO:guidedlda:<120> log likelihood: -4228842
INFO:guidedlda:<140> log likelihood: -4228269
INFO:guidedlda:<160> log likelihood: -4227467
INFO:guidedlda:<180> log likelihood: -4226904
INFO:guidedlda:<199> log likelihood: -4227882


...

(4, 202)
['moneygram', 'sent', 'send', 'transaction', 'phone', 'gram', 'account', 'number']
(4, 202)
['paypal', 'account', 'credit', 'payment', 'funds', 'transaction', 'card', 'bank']
(4, 202)
['bank', 'account', 'transfer', 'wire', 'funds', 'sent', '2015', 'number']
(4, 202)
['union', 'western', 'wu', 'transfer', 'transaction', 'bank', 'number', 'received']


INFO:guidedlda:<20> log likelihood: -21216
INFO:guidedlda:<40> log likelihood: -20870
INFO:guidedlda:<60> log likelihood: -20676
INFO:guidedlda:<80> log likelihood: -20608
INFO:guidedlda:<100> log likelihood: -20595
INFO:guidedlda:<120> log likelihood: -20599
INFO:guidedlda:<140> log likelihood: -20493
INFO:guidedlda:<160> log likelihood: -20395
INFO:guidedlda:<180> log likelihood: -20460
INFO:guidedlda:<199> log likelihood: -20347


(4, 187)
['money', 'order', 'bank', 'fee', 'account', 'charge', 'charged', 'pay']
(4, 187)
['credit', 'debt', 'company', 'payments', 'contacted', 'money', 'fee', 'fees']
(4, 187)
['account', 'money', 'received', 'payment', 'loan', 'company', 'mortgage', 'bank']
(4, 187)
['check', 'account', 'cash', 'bank', 'checks', 'cashed', 'request', 'person']


INFO:guidedlda:n_documents: 872
INFO:guidedlda:vocab_size: 165
INFO:guidedlda:n_words: 23910
INFO:guidedlda:n_topics: 4
INFO:guidedlda:n_iter: 200
INFO:guidedlda:<0> log likelihood: -164266
INFO:guidedlda:<20> log likelihood: -124686
INFO:guidedlda:<40> log likelihood: -121517
INFO:guidedlda:<60> log likelihood: -120474
INFO:guidedlda:<80> log likelihood: -120626
INFO:guidedlda:<100> log likelihood: -120357
INFO:guidedlda:<120> log likelihood: -120234
INFO:guidedlda:<140> log likelihood: -120111
INFO:guidedlda:<160> log likelihood: -119878
INFO:guidedlda:<180> log likelihood: -119684
INFO:guidedlda:<199> log likelihood: -119941


(4, 165)
['account', 'money', 'funds', 'check', 'days', 'rush', 'access', 'netspend']
(4, 165)
['account', 'money', 'days', 'received', 'time', 'service', 'dispute', 'customer']
(4, 165)
['cards', 'express', 'account', 'american', 'number', 'bank', 'money', 'credit']
(4, 165)
['money', 'rushcard', 'account', 'access', 'deposit', 'funds', 'direct', 'rush']


In [96]:
c

array(['Debt collection', 'Consumer Loan', 'Mortgage', 'Credit card',
       'Credit reporting', 'Student loan', 'Bank account or service',
       'Payday loan', 'Money transfers', 'Other financial service',
       'Prepaid card'], dtype=object)

In [97]:
for k,i in enumerate(classTopics):
    print('\nTopic:  '+c[k])
    for u in i:
        print(u)


Topic:  Debt collection
['credit', 'account', 'report', 'company', 'information', 'letter', 'received', 'sent']
['phone', 'number', 'company', 'pay', 'payment', 'received', 'account', 'time']
['credit', 'account', 'report', 'letter', 'company', 'received', 'sent', 'information']
['credit', 'company', 'phone', 'number', 'account', 'received', 'paid', 'information']

Topic:  Consumer Loan
['payment', 'credit', 'car', 'payments', 'vehicle', 'paid', 'account', 'pay']
['payment', 'account', 'bank', 'credit', 'received', 'late', 'pay', 'payments']
['credit', 'vehicle', 'car', 'account', 'report', 'information', 'received', 'company']
['car', 'company', 'payment', 'pay', 'credit', 'payments', 'time', 'paid']

Topic:  Mortgage
['loan', 'wells', 'fargo', 'home', 'sale', 'property', 'foreclosure', 'modification']
['loan', 'modification', 'bank', 'payment', 'home', 'payments', 'chase', 'credit']
['payment', 'account', 'payments', 'escrow', 'loan', 'received', 'insurance', 'paid']
['loan', 'home'